In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

顶点AI管道：评估AutoML表格分类模型的BatchPrediction结果

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_classification_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_classification_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_classification_model_evaluation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本笔记本演示了如何使用Vertex AI分类模型评估组件来评估AutoML分类模型。模型评估帮助您根据评估指标确定模型的性能，并在必要时改进模型。

###目标

在本教程中，您将训练一个Vertex AI AutoML Tabular Classification模型，并学习如何通过Vertex AI管道作业使用`google_cloud_pipeline_components`对其进行评估：

本教程使用以下Google Cloud ML服务和资源:

- Vertex AI `数据集(Datasets)`
- Vertex AI `训练`(AutoML Tabular Classification) 
- Vertex AI `模型注册表`(Model Registry)
- Vertex AI `管道`(Pipelines)
- Vertex AI `批量预测`(Batch Predictions)

执行的步骤包括:

- 创建一个Vertex AI `数据集(Dataset)`。
- 在`数据集(Dataset)`资源上训练AutoML Tabular Classification模型。
- 将训练的`AutoML模型资源`导入到管道中。
- 运行一个`批量预测(Batch Prediction)`作业。
- 使用`分类评估组件(Classification Evaluation Component)`评估AutoML模型。
- 将分类指标导入到AutoML模型资源中。

### 数据集

此笔记本中使用的数据集是PetFinder数据集的一部分，可在Kaggle上的[此处](https://www.kaggle.com/c/petfinder-adoption-prediction)找到。当前数据集仅是用于预测宠物是否被领养的原数据集的一部分。它包含以下字段：

- `Type`：动物类型（1 = 狗，2 = 猫）
- `Age`：列出时宠物的年龄，以月计
- `Breed1`：宠物的主要品种
- `Gender`：宠物的性别
- `Color1`：宠物的颜色1
- `Color2`：宠物的颜色2
- `MaturitySize`：成熟时的大小（1 = 小型，2 = 中型，3 = 大型，4 = 特大型，0 = 未指定）
- `FurLength`：毛长（1 = 短毛，2 = 中毛，3 = 长毛，0 = 未指定）
- `Vaccinated`：宠物是否接种疫苗（1 = 是，2 = 否，3 = 不确定）
- `Sterilized`：宠物是否绝育（1 = 是，2 = 否，3 = 不确定）
- `Health`：健康状况（1 = 健康，2 = 轻微受伤，3 = 严重受伤，0 = 未指定）
- `Fee`：领养费用（0 = 免费）
- `PhotoAmt`：为该宠物上传的照片总数
- `Adopted`：宠物是否被领养（是/否）。

**注意**：此数据集已被移动到公共云存储桶中，从中在此笔记本中访问。

成本
本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI的价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage的价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)根据您预计的使用量生成成本估算。

### 设置本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，那么您的环境已满足运行此笔记本的所有要求。

否则，请确保您的环境满足本笔记本的要求。 您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南[设置Python开发环境](https://cloud.google.com/python/setup) 和[Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。 以下步骤提供了一个简化的指令集：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

1. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

1. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。 激活虚拟环境。

1. 要安装Jupyter，请在终端shell中的命令行上运行`pip3 install jupyter`。

1. 要启动Jupyter，请在终端shell中的命令行上运行`jupyter notebook`。

1. 在Jupyter Notebook Dashboard中打开这个笔记本。

安装

安装以下需要执行此笔记本的包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install google-cloud-pipeline-components==1.0.17 {USER_FLAG} -q
! pip3 install --upgrade kfp google-cloud-pipeline-components {USER_FLAG} -q
! pip3 install --upgrade matplotlib {USER_FLAG} -q

重新启动内核

在安装了额外的软件包之后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

### 设置您的Google Cloud项目

**不论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用，用于支付计算/存储成本。

1. [确保您的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI、Compute Engine和Dataflow API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,dataflow.googleapis.com)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter会将以`!`前缀开头的行解释为shell命令，并会将以`$`前缀开头的Python变量插入这些命令中。

####设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您也可以更改“REGION”变量，该变量用于笔记本其余部分的操作。以下是 Vertex AI 支持的地区。建议选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多地区存储桶进行 Vertex AI 的训练。并非所有地区都支持所有的 Vertex AI 服务。

了解有关[Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行实时教程会话，可能会使用共享测试账户或项目。为避免用户在创建的资源之间发生名称冲突，您可以为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云账户

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经经过身份验证。请跳过这一步。

如果您正在使用Colab，请运行下面的单元格并按照提示进行身份验证以通过oAuth验证您的帐号。

**否则**，请按照以下步骤操作：

1. 在Cloud Console中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入名称，然后点击**创建**。

4. 在**将此服务帐号授予项目访问权限**部分，点击**角色**下拉列表。在过滤框中键入"Vertex AI"，然后选择**Vertex AI管理员**。在过滤框中键入"存储对象管理员"，然后选择**存储对象管理员**。

5. 点击**创建**。包含您密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中输入您的服务帐号密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，都需要执行以下步骤。**

当您使用Cloud SDK运行Vertex AI管道作业时，作业会将管道工件存储在一个云存储桶中。在本教程中，您将创建一个Vertex AI管道作业，将评估指标和特征属性等工件保存到一个云存储桶中。

在下方设置您的云存储桶的名称。它必须在所有云存储桶中保持唯一。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查存储桶的内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

服务账号

您可以使用服务帐号创建Vertex AI Pipeline作业。如果您不想使用项目的Compute Engine服务帐号，请将`SERVICE_ACCOUNT`设置为另一个服务帐号ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为Vertex AI Pipelines设置服务账号访问权限

运行以下命令，为您的服务账号授予权限，以读取和写入在前面创建的存储桶中的管道工件。您只需要在每个服务账号上运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库

导入 Vertex AI Python SDK 和其他所需的 Python 库。

In [ ]:
import json

import google.cloud.aiplatform as aiplatform
import kfp
import matplotlib.pyplot as plt
from google.cloud import aiplatform_v1
from kfp.v2 import compiler

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 版本的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

创建顶点AI数据集

使用数据集来源在顶点AI中创建一个管理的表格数据集资源。

In [ ]:
DATA_SOURCE = "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/petfinder-tabular-classification.csv"

In [ ]:
# Create the Vertex AI Dataset resource
dataset = aiplatform.TabularDataset.create(
    display_name="petfinder-tabular-dataset_" + UUID,
    gcs_source=DATA_SOURCE,
)

print("Resource name:", dataset.resource_name)

训练AutoML模型

使用“Adopted”作为目标列，对创建的数据集训练一个简单的分类模型。

设置显示名称，并使用`AutoMLTabularTrainingJob`创建训练任务，为列转换指定适当的数据类型。

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "[your-train-job-display-name]"  # @param {type:"string"}

In [ ]:
# If no display name is specified, use the default one
if (
    TRAINING_JOB_DISPLAY_NAME == ""
    or TRAINING_JOB_DISPLAY_NAME is None
    or TRAINING_JOB_DISPLAY_NAME == "[your-train-job-display-name]"
):
    TRAINING_JOB_DISPLAY_NAME = "train-petfinder-automl_" + UUID

`AutoMLTabularTrainingJob` 类使用以下参数创建一个 AutoML 训练作业：

- `display_name`：Vertex AI 训练作业资源的人类可读名称。
- `optimization_prediction_type`：模型要产生的预测类型。例如：回归，分类。
- `column_specs`（可选）：要应用于输入列的转换（包括数据类型更正）。
- `optimization_objective`：要最小化或最大化的优化目标。根据预测类型的不同，选择此参数。如果未设置此字段，则使用默认优化目标函数。

有关更多详细信息，请查看[**AutoMLTabularTrainingJob**类的文档](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.AutoMLTabularTrainingJob)。

In [ ]:
# Define the AutoML training job
train_job = aiplatform.AutoMLTabularTrainingJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    optimization_prediction_type="classification",
    column_specs={
        "Type": "categorical",
        "Age": "numeric",
        "Breed1": "categorical",
        "Color1": "categorical",
        "Color2": "categorical",
        "MaturitySize": "categorical",
        "FurLength": "categorical",
        "Vaccinated": "categorical",
        "Sterilized": "categorical",
        "Health": "categorical",
        "Fee": "numeric",
        "PhotoAmt": "numeric",
    },
)

设置模型的显示名称。

In [ ]:
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

In [ ]:
# If no name is specified, use the default name
if (
    MODEL_DISPLAY_NAME == ""
    or MODEL_DISPLAY_NAME is None
    or MODEL_DISPLAY_NAME == "[your-model-display-name]"
):
    MODEL_DISPLAY_NAME = "pet-adoption-prediction-model_" + UUID

在创建的TabularDataset上运行训练作业，通过传递以下参数进行训练：

- `dataset`：需要用于训练模型的同一项目中的TabularDataset数据。
- `target_column`：要预测的列值的名称。
- `model_display_name`：生成的Vertex AI Model的显示名称。
- `budget_milli_node_hours`（可选）：创建此模型的训练预算，以毫节点小时表示，即此字段中的值为1,000表示1个节点小时。模型的训练成本不超过此预算。

有关在`run()`方法中使用的其他参数的更多详细信息，请访问此[ **AutoMLTabularTrainingJob** 类文档](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.AutoMLTabularTrainingJob#google_cloud_aiplatform_AutoMLTabularTrainingJob_run)。

训练作业大约需要1.5-2小时才能完成。

In [ ]:
# Specify the target column
target_column = "Adopted"

# Run the training job
model = train_job.run(
    dataset=dataset,
    target_column=target_column,
    model_display_name=MODEL_DISPLAY_NAME,
    budget_milli_node_hours=1000,
)

## 从训练中列出模型评估

在训练作业完成后，获取模型评估并打印出来。

In [ ]:
# Get evaluations
model_evaluations = model.list_model_evaluations()

model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

In [ ]:
# Print the evaluation metrics
for evaluation in model_evaluations:
    evaluation = evaluation.to_dict()
    print("Model's evaluation metrics from Training:\n")
    metrics = evaluation["metrics"]
    for metric in metrics.keys():
        print(f"metric: {metric}, value: {metrics[metric]}\n")

## 为评估创建流水线

现在，您可以运行一个 Vertex AI BatchPrediction 作业，并生成对其结果的评估和特征归因。

为此，您可以使用[`google-cloud-pipeline-components`](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/index.html) Python 包中提供的组件，创建一个 Vertex AI 流水线。

### 定义流水线

在定义流水线的流程中，首先获取模型资源。然后，从提供的源数据集中对批量预测进行采样并创建一个批量预测。在创建批量预测作业时启用解释以生成特征归因。完成批量预测作业后，从结果中获取分类评估指标和特征归因。

该流水线使用以下组件：

- `GetVertexModelOp`：获取一个 Vertex 模型工件。更多详细信息，请参见 [此处](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.experimental.evaluation.html#google_cloud_pipeline_components.experimental.evaluation.GetVertexModelOp)。
- `EvaluationDataSamplerOp`：随机对输入数据集进行下采样，以用于计算用于 AutoML Tables 和自定义模型的 Vertex XAI 特征归因。使用 Apache Beam 创建一个 Dataflow 作业来对数据集进行下采样。更多详细信息，请参见 [此处](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.experimental.evaluation.html#google_cloud_pipeline_components.experimental.evaluation.EvaluationDataSamplerOp)。
- `ModelBatchPredictOp`：创建一个 Google Cloud Vertex BatchPredictionJob 并等待其完成。更多详细信息，请参见 [此处](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.aiplatform.html#google_cloud_pipeline_components.aiplatform.ModelBatchPredictOp)。
- `ModelEvaluationClassificationOp`：在训练模型的批量预测结果上计算评估指标。使用 Apache Beam 和 TFMA 创建一个 Dataflow 作业来计算评估指标。支持表格、图像、视频和文本数据的多类别分类评估。更多详细信息，请参见 [此处](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.experimental.evaluation.html#google_cloud_pipeline_components.experimental.evaluation.ModelEvaluationClassificationOp)。
- `ModelEvaluationFeatureAttributionOp`：在训练模型的批量解释结果上计算特征归因。使用 Apache Beam 和 TFMA 创建一个 Dataflow 作业来计算特征归因。更多详细信息，请参见 [此处](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.experimental.evaluation.html#google_cloud_pipeline_components.experimental.evaluation.ModelEvaluationFeatureAttributionOp)。
- `ModelImportEvaluationOp`：使用 ModelService.ImportModelEvaluation 将模型评估工件导入到现有的 Vertex 模型中。更多详细信息，请参见 [此处](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/google_cloud_pipeline_components.experimental.evaluation.html#google_cloud_pipeline_components.experimental.evaluation.ModelImportEvaluationOp)。

In [ ]:
@kfp.dsl.pipeline(
    name="vertex-evaluation-automl-tabular-classification-feature-attribution"
)
def evaluation_automl_tabular_feature_attribution_pipeline(
    project: str,
    location: str,
    root_dir: str,
    model_name: str,
    target_column_name: str,
    batch_predict_gcs_source_uris: list,
    batch_predict_instances_format: str,
    batch_predict_predictions_format: str = "jsonl",
    batch_predict_machine_type: str = "n1-standard-4",
    batch_predict_explanation_metadata: dict = {},
    batch_predict_explanation_parameters: dict = {},
    batch_predict_explanation_data_sample_size: int = 10000,
):

    from google_cloud_pipeline_components.aiplatform import ModelBatchPredictOp
    from google_cloud_pipeline_components.experimental.evaluation import (
        EvaluationDataSamplerOp, GetVertexModelOp,
        ModelEvaluationClassificationOp, ModelEvaluationFeatureAttributionOp,
        ModelImportEvaluationOp)

    # Get the Vertex AI model resource
    get_model_task = GetVertexModelOp(model_resource_name=model_name)

    # Run Data-sampling task
    data_sampler_task = EvaluationDataSamplerOp(
        project=project,
        location=location,
        root_dir=root_dir,
        gcs_source_uris=batch_predict_gcs_source_uris,
        instances_format=batch_predict_instances_format,
        sample_size=batch_predict_explanation_data_sample_size,
    )

    # Run Batch Explanations
    batch_explain_task = ModelBatchPredictOp(
        project=project,
        location=location,
        model=get_model_task.outputs["model"],
        job_display_name="model-registry-batch-predict-evaluation",
        gcs_source_uris=data_sampler_task.outputs["gcs_output_directory"],
        instances_format=batch_predict_instances_format,
        predictions_format=batch_predict_predictions_format,
        gcs_destination_output_uri_prefix=root_dir,
        machine_type=batch_predict_machine_type,
        # Set the explanation parameters
        generate_explanation=True,
        explanation_parameters=batch_predict_explanation_parameters,
        explanation_metadata=batch_predict_explanation_metadata,
    )

    # Run evaluation based on prediction type and feature attribution component.
    # After, import the model evaluations to the Vertex model.
    eval_task = ModelEvaluationClassificationOp(
        project=project,
        location=location,
        root_dir=root_dir,
        problem_type="classification",
        ground_truth_column=target_column_name,
        predictions_gcs_source=batch_explain_task.outputs["gcs_output_directory"],
        predictions_format=batch_predict_predictions_format,
    )

    # Get Feature Attributions
    feature_attribution_task = ModelEvaluationFeatureAttributionOp(
        project=project,
        location=location,
        root_dir=root_dir,
        predictions_format=batch_predict_predictions_format,
        predictions_gcs_source=batch_explain_task.outputs["gcs_output_directory"],
    )

    ModelImportEvaluationOp(
        classification_metrics=eval_task.outputs["evaluation_metrics"],
        feature_attributions=feature_attribution_task.outputs["feature_attributions"],
        model=get_model_task.outputs["model"],
        dataset_type=batch_predict_instances_format,
    )

### 编译管道

接下来，将管道编译到 `tabular_classification_pipeline.json` 文件中。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=evaluation_automl_tabular_feature_attribution_pipeline,
    package_path="tabular_classification_pipeline.json",
)

定义运行管道的参数

指定运行管道所需的参数。

为您的管道设置一个显示名称。

In [ ]:
PIPELINE_DISPLAY_NAME = "[your-pipeline-display-name]"  # @param {type:"string"}

In [ ]:
# If no display name is set, use the default one
if (
    PIPELINE_DISPLAY_NAME == "[your-pipeline-display-name]"
    or PIPELINE_DISPLAY_NAME == ""
    or PIPELINE_DISPLAY_NAME is None
):
    PIPELINE_DISPLAY_NAME = "pet_adoption_" + UUID

为了将所需参数传递给流水线，您定义以下参数如下：

- `project`：项目ID。
- `location`：运行流水线的地区。
- `root_dir`：用于保存暂存文件和文件的GCS目录。在该目录下创建一个随机子目录，用于保留作业信息，以便在发生故障时恢复作业。
- `model_name`：经过训练的AutoML表格分类模型的资源名称。
- `target_column_name`：用于分类的目标列的名称。
- `batch_predict_gcs_source_uris`：批量预测输入实例的Cloud Storage桶uri列表。
- `batch_predict_instances_format`：用于批量预测的输入实例的格式。可以是 **jsonl**、**bigquery** 或 **csv**。
- `batch_predict_explanation_data_sample_size`：用于批量预测和评估的样本大小。

In [ ]:
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/pet_adoption_{UUID}"
parameters = {
    "project": PROJECT_ID,
    "location": REGION,
    "root_dir": PIPELINE_ROOT,
    "model_name": model.resource_name,
    "target_column_name": "Adopted",
    "batch_predict_gcs_source_uris": [DATA_SOURCE],
    "batch_predict_instances_format": "csv",
    "batch_predict_explanation_data_sample_size": 3000,
}

使用以下参数创建一个 Vertex AI 管道作业：

- `display_name`：管道的名称，将显示在 Google Cloud 控制台中。
- `template_path`：PipelineJob 或 PipelineSpec JSON 或 YAML 文件的路径。可以是本地路径、Google Cloud Storage URI 或 Artifact Registry URI。
- `parameter_values`：从运行时参数名称到控制管道运行的值的映射。
- `enable_caching`：是否为运行启用缓存。

从[此文档](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.PipelineJob)中了解有关 `PipelineJob` 类的更多信息。

创建后，使用配置的 `SERVICE_ACCOUNT` 运行管道作业。

In [ ]:
job = aiplatform.PipelineJob(
    display_name=PIPELINE_DISPLAY_NAME,
    template_path="tabular_classification_pipeline.json",
    parameter_values=parameters,
    enable_caching=True,
)

job.run(service_account=SERVICE_ACCOUNT)

模型评估

在上一个步骤的结果中，点击生成的链接查看在云控制台中运行的情况。

在用户界面中，当您点击时，许多管道有向无环图（DAG）节点会展开或折叠。以下是DAG的部分展开视图（点击图像查看更大版本）。

<img src="images/automl_tabular_classification_evaluation_pipeline.PNG">

获取模型评估结果

在评估流程完成后，运行下面的单元格以打印评估指标。

In [ ]:
# Iterate over the pipeline tasks
for task in job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the evaluation task
    if (
        ("model-evaluation" in task.task_name)
        and ("model-evaluation-import" not in task.task_name)
        and (
            task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
            or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
        )
    ):
        evaluation_metrics = task.outputs.get("evaluation_metrics").artifacts[0]
        evaluation_metrics_gcs_uri = evaluation_metrics.uri

print(evaluation_metrics)
print(evaluation_metrics_gcs_uri)

### 可视化指标

使用柱状图来可视化可用的指标，如`auRoc`和`logLoss`。

In [ ]:
metrics = []
values = []
for i in evaluation_metrics.metadata.items():
    metrics.append(i[0])
    values.append(i[1])
plt.figure(figsize=(5, 3))
plt.bar(x=metrics, height=values)
plt.title("Evaluation Metrics")
plt.ylabel("Value")
plt.show()

获取特征归因

运行以下单元格以获取特征归因。

In [ ]:
# Iterate over the pipeline tasks
for task in job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the feature attribution task
    if (task.task_name == "feature-attribution") and (
        task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
        or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
    ):
        feat_attrs = task.outputs.get("feature_attributions").artifacts[0]
        feat_attrs_gcs_uri = feat_attrs.uri

print(feat_attrs)
print(feat_attrs_gcs_uri)

从获取到的功能归因的云存储URI中获取归因值。

In [ ]:
# Load the results
attributions = !gsutil cat $feat_attrs_gcs_uri

# Print the results obtained
attributions = json.loads(attributions[0])
print(attributions)

### 可视化特征归因

使用直方图可视化每个特征的归因。

In [ ]:
data = attributions["explanation"]["attributions"][0]["featureAttributions"]
features = []
attr_values = []
for key, value in data.items():
    features.append(key)
    attr_values.append(value)

plt.figure(figsize=(5, 3))
plt.bar(x=features, height=attr_values)
plt.title("Feature Attributions")
plt.xticks(rotation=90)
plt.ylabel("Attribution value")
plt.show()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

将`delete_bucket`设置为**True**以创建在本笔记本中创建的Cloud Storage存储桶。

In [ ]:
# Delete model resource
model.delete()

# Delete the dataset resource
dataset.delete()

# Delete the training job
train_job.delete()

# Delete the evaluation pipeline
job.delete()

# Delete Cloud Storage objects
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI